# Hierchies in the student network

## Preamble

In [ ]:
#from sensible_raw.loaders import loader #This loader can onlu be used if you have access to the compute server
import pandas as pd
import numpy as np
import calendar
import re
import seaborn as sns
import networkx as nx
import netwulf as nw
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
import pandas as pd
#!pip install hvplot
import holoviews as hv
import networkx as nx
from holoviews import opts
import hvplot.networkx as hvnx
#from bokeh.models import HoverTool
from network_functions import *
from functools import reduce

In [ ]:
all_periods = [
    "%s_%s" % (month.lower(), year)
    for year in ["2012","2013", "2014", "2015", "2016"]
    for month in [calendar.month_name[m] for m in range(1, 13)]
]

grades = get_datatype_period("grades", all_periods, sort=True)
calls = get_datatype_period("calllog", all_periods, sort=True)
sms = get_datatype_period("sms", all_periods, sort=True)
#gender = get_datatype_period("user_metadata", all_periods, sort=True)

create_nw_csv(sms, 'sms')
create_nw_csv(calls, 'call')

In [ ]:
#This data will not be available as it would against the contract of not distributing the data.
call = pd.read_csv("call_network.csv")
sms = pd.read_csv("sms_network.csv")

all_dfs = [sms, call]
combined = pd.concat(all_dfs).reset_index(drop=True)
combined = combined.drop(['Unnamed: 0'], axis=1)

#Initializing networks
call_nw = initialize_network(call, "user", "address")
sms_nw = initialize_network(sms, "user", "address")
combined_nw = initialize_network(combined, "user", "address")

In [2]:
#get_stats(call_nw)
#get_stats(sms_nw)
#get_stats(combined_nw)

Amount of nodes in network: 784 
Amount of edges in network: 3771
The probability that two nodes are connected: 		 p = 1.228595 %
The average degree of the nodes are: 			 <k> = 9.6199
Standard deviation of degrees of the network:	 9.2929
Max degree: 85
Min degree: 0
Max In degree: 39
Min In degree: 0
Max Out degree: 46
Min Out degree: 0
Dangling Nodes: 61 

Amount of nodes in network: 795
Amount of edges in network: 3788
The probability that two nodes are connected: 		 p = 1.200196 %
The average degree of the nodes are: 			 <k> = 9.5296
Standard deviation of degrees of the network:	 8.7462
Max degree: 73
Min degree: 0
Max In degree: 34
Min In degree: 0
Max Out degree: 39
Min Out degree: 0
Dangling Nodes: 50 

Amount of nodes in network: 807
Amount of edges in network: 4674
The probability that two nodes are connected: 		 p = 1.437177 %
The average degree of the nodes are: 			 <k> = 11.5836
Standard deviation of degrees of the network:	 11.0308
Max degree: 95
Min degree: 0
Max In degree: 44
Min In degree: 0
Max Out degree: 51
Min Out degree: 0
Dangling Nodes: 51 

In [ ]:
#Would visualise the networks with PageRank as colour and the sprin layout.
visualise_network_spring(sms_nw)
visualise_network_spring(call_nw)
visualise_network_spring(combined_nw)

In [ ]:
#Creates an edgelist from the dataframe 
create_edgelist(sms, thres=20)

In [ ]:
#Creates a rankings dataframe from an edgelist and colors the network in respect to ranking
rankings = rankings_df("df_edgelist_thres20")
G = sms_nw

visualise_network_ranking(G, rankings, size_scale=100)

In [ ]:
#Shows the correlation heatmap
correlation_heatmap(sms_nw, rankings)

In [ ]:
#Plots the distribution of ranks
df = rankings
sns.histplot(df[df['Rank']>0].Rank, bins=int(20), kde=True).set_title('Distribution of Ranks')
plt.show()

## Mean statitics between males and females

In [ ]:
df = network_ranking_df(sms_nw, rankings)
print(df[df['gender']=='M'].mean())
print(df[df['gender']=='K'].mean())

Males: <br>
degree             12.788079 <br>
Participant       351.950331 <br>
in_out                   inf <br>
grade               7.281222 <br>
semester            4.384106 <br>
starting_year    2012.470199 <br>
Pagerank            0.001498 <br>
Rank               80.336082 <br>

Females: <br>
degree             15.141509 <br>
Participant       375.056604 <br>
in_out                   inf <br>
grade               7.089734 <br>
semester            4.405660 <br>
starting_year    2012.594340 <br>
Pagerank            0.001695 <br>
Rank               90.470649 <br>

## Does Ranking change over time?

### For rank

In [ ]:
create_time_csv(smsraw, "sms")

numbers = [3,4,5] #Chosen because it contains the semesters with most data on students 
dfs = []
for i in numbers:
    rankings = rankings_df("time/sms_{}_edgelist".format(i))
    rankings = rankings.rename(columns={"Rank": "Rank_{}".format(i)})
    rankings = rankings.drop(['Rank_norm'], axis=1)
    dfs.append(rankings)
    
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Participant'], how='outer'), dfs)
df_merged = df_merged.dropna()
df_merged = df_merged.drop(['Participant'], axis=1) #Drop nans and unnecessary columns

In [ ]:
#Normalise the ranks
df_merged['Rank_3'] = df_merged['Rank_3'].div(max(df_merged['Rank_3']))
df_merged['Rank_4'] = df_merged['Rank_4'].div(max(df_merged['Rank_4']))
df_merged['Rank_5'] = df_merged['Rank_5'].div(max(df_merged['Rank_5']))

In [ ]:
#Plot the rank
ax = df_merged.mean().plot.line(linewidth=5)
df_merged.T.plot.line(alpha=0.2, legend=None, color='lightblue', ax=ax)
plt.show

In [ ]:
#Print the difference
df_merged['diff34'] = df_merged['Rank_4']-df_merged['Rank_3']
df_merged['diff45'] = df_merged['Rank_5']-df_merged['Rank_4']

print(df_merged.diff34.mean())
df_merged.diff45.mean()

### Pagerank

In [ ]:
df3 = pd.read_csv('time/sms_3_network.csv')
df4 = pd.read_csv('time/sms_4_network.csv')
df5 = pd.read_csv('time/sms_5_network.csv')

#Create networks
nw3 = nx.from_pandas_edgelist(df3, source='user', target='address', edge_attr=None, create_using = nx.DiGraph())
nw4 = nx.from_pandas_edgelist(df4, source='user', target='address', edge_attr=None, create_using = nx.DiGraph())
nw5 = nx.from_pandas_edgelist(df5, source='user', target='address', edge_attr=None, create_using = nx.DiGraph())

#Calculate PageRank
pr3 = nx.pagerank(nw3, alpha=0.65)
pr4 = nx.pagerank(nw4, alpha=0.65)
pr5 = nx.pagerank(nw5, alpha=0.65)

prdf3 = pd.DataFrame.from_dict(pr3, orient='index', columns=["Pagerank_3"])
prdf4 = pd.DataFrame.from_dict(pr4, orient='index', columns=["Pagerank_4"])
prdf5 = pd.DataFrame.from_dict(pr5, orient='index', columns=["Pagerank_5"])

#Normalise
prdf3["Pagerank_3"] = prdf3["Pagerank_3"].div(max(prdf3['Pagerank_3']))
prdf4["Pagerank_4"] = prdf4["Pagerank_4"].div(max(prdf4['Pagerank_4']))
prdf5["Pagerank_5"] = prdf5["Pagerank_5"].div(max(prdf5['Pagerank_5']))

prdf3['Participant'] = prdf3.index
prdf4['Participant'] = prdf4.index
prdf5['Participant'] = prdf5.index

data_frames = [prdf3, prdf4, prdf5]
#Combine to one Dataframe
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Participant'], how='outer'), data_frames)
df_merged = df_merged.drop(['Participant'], axis=1)

In [ ]:
#plot and print differnce
ax = df_merged.mean().plot.line(linewidth=5)
df_merged.T.plot.line(alpha=0.2, legend=None, color='lightblue', ax=ax)
plt.show

In [ ]:
df_merged['diff34'] = df_merged['Pagerank_4']-df_merged['Pagerank_3']
df_merged['diff45'] = df_merged['Pagerank_5']-df_merged['Pagerank_4']

print(df_merged.diff34.mean())
df_merged.diff45.mean()

## In and out networks

In [ ]:
visualise_in_out_networks(sms, ranking) #check functions.py

## XGBoost Prediction

In [ ]:
#!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error

#ranking = rankings_df("df_edgelist_thres20")
data = network_ranking_df(sms_nw, ranking)
#data = data[data["Rank"]>-1]
X = data.drop(['in_out', 'Participant', 'Pagerank', 'Rank', 'degree', 'semester'], axis=1).dropna().copy()
y = data['Rank'].copy()

# Import DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# Convert df into a dictionary using .to_dict(): df_dict
df_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False)

# Apply dv on df_dict: df_encoded
X_encoded = dv.fit_transform(df_dict)

X_encoded = pd.DataFrame(X_encoded)

X_train, X_test, y_train, y_test= train_test_split(X_encoded, y, test_size=0.2, random_state=42069)

In [ ]:
#Initial model
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)

print(mse)
print(rmse)

6787.895624946226 <br>
82.38868626787435

In [ ]:
#Parameter search
param = {'max_depth': [3,4], 
         'learning_rate': [0.01, 0.1], 
         'gamma':[0.25, 1],
         'reg_lambda':[5.0, 10.0],
         'scale_pos_weight':[1,5]}

optimal_params = GridSearchCV(
         estimator=xgb.XGBRegressor(
         objective='reg:squarederror', seed=42069, subsample=0.9),
         param_grid=param, cv=3, verbose=5).fit(x_train,y_train)
          
# optimal parameters 
print(optimal_params.best_params_) 

{'gamma': 0.25, 'learning_rate': 0.1, 'max_depth': 3, 'reg_lambda': 10.0, 'scale_pos_weight': 1}

In [ ]:
param = {'max_depth': [2,3], 
         'learning_rate': [0.1], 
         'gamma':[0, 0.25],
         'reg_lambda':[10.0, 15.0],
         'scale_pos_weight':[0.5,1]}

optimal_params = GridSearchCV(
         estimator=xgb.XGBRegressor(
         objective='reg:squarederror', seed=42069, subsample=0.9),
         param_grid=param, cv=3, verbose=5).fit(x_train,y_train)
          
# optimal parameters 
print(optimal_params.best_params_) 

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'reg_lambda': 15.0, 'scale_pos_weight': 0.5}

In [ ]:
param = {'max_depth': [2], 
         'learning_rate': [0.1, 0.2], 
         'gamma':[0],
         'reg_lambda':[15.0, 25.0],
         'scale_pos_weight':[0.5]}

optimal_params = GridSearchCV(
         estimator=xgb.XGBRegressor(
         objective='reg:squarederror', seed=42069, subsample=0.9),
         param_grid=param, cv=3, verbose=5).fit(x_train,y_train)
          
# optimal parameters 
print(optimal_params.best_params_)

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'reg_lambda': 25.0, 'scale_pos_weight': 0.5}

In [ ]:
param = {'max_depth': [1,2], 
         'learning_rate': [0.1], 
         'gamma':[0],
         'reg_lambda':[25.0, 30.0, 35.0],
         'scale_pos_weight':[0.5]}

optimal_params = GridSearchCV(
         estimator=xgb.XGBRegressor(
         objective='reg:squarederror', seed=42069, subsample=0.9),
         param_grid=param, cv=3, verbose=5).fit(x_train,y_train)
          
# optimal parameters 
print(optimal_params.best_params_)

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 1, 'reg_lambda': 25.0, 'scale_pos_weight': 0.5}

In [ ]:
final_model = xgb.XGBRegressor(seed=42069, objective='reg:squarederror', max_depth=1, learning_rate=0.1, gamma=0,reg_lambda=25.0, scale_pos_weight=0.5)
final_model.fit(X_train,y_train,verbose=True, eval_metric='rmse')

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(final_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) ) 

scores_final = cross_val_score(final_model, X_train, y_train, cv=10)
print("Mean cross-validation score: %.2f" % scores_final.mean())

y_pred_final = final_model.predict(X_test)

mse_final = mean_squared_error(y_test, y_pred_final)
print("FINAL MSE: %.2f" % mse_final)

# Root mean squared error
rmse_final= np.sqrt(mse_final)
print("FINAL RMSE: %.2f" % rmse_final)

#plots the feature importance
xgb.plot_importance(final_model)
import pprint
pprint.pprint(dv.vocabulary_)

## Do we connect with people of similar grade?

In [ ]:
grades = pd.read_csv('GPA.csv')
grades
grades = grades[(grades.user > -1) & (grades.user < 853)]

In [ ]:
grade_texter = []
grade_reciever = []
for i in range(len(sms)):
    try:
        grade_texter.append(grades[grades['user'] == sms['user'][i]].grade.values.tolist()[0])
    except IndexError:
        grade_texter.append(np.nan)
    try:
        grade_reciever.append(grades[grades['user'] == sms['address'][i]].grade.values.tolist()[0])
    except IndexError:
        grade_reciever.append(np.nan)

In [ ]:
data = {'texter':sms['user'],
        'reciever':sms['address'],
        'sent':sms['sent'],
        'texter_grade':grade_texter,
        'reciever_grade':grade_reciever,
        'grade_difference': abs(np.array(grade_texter)-np.array(grade_reciever))}

dataf = pd.DataFrame(data)
dataf.grade_difference.mean()

2.4037718218695465

In [ ]:
import random
a = sms['user'].values.tolist()
b = grade_texter

x = sms['address'].values.tolist()
y = grade_reciever

c = list(zip(a, b))
z = list(zip(x, y))

means = []

for i in range(1000):
    random.shuffle(c)
    random.shuffle(z)

    a, b = zip(*c)
    x, y = zip(*z)

    means.append(np.nanmean(abs(np.array(b)-np.array(y))))

np.mean(means)

2.8586856652713877

In [ ]:
small_to_big = 0
big_to_small = 0
for i in range(len(dataf)):
    if dataf['texter_grade'][i] < dataf['reciever_grade'][i]:
        small_to_big += 1
    else:
        big_to_small += 1

print(small_to_big)
print(big_to_small)

1733 <br>
2403